In [0]:
%tensorflow_version 2.x

In [27]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2020-04-24 13:13:20--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2020-04-24 13:13:21--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc140f390f5f3e1ba8efe93669d.dl.dropboxusercontent.com/zip_download_get/AZ2B77jCErqrL0IC_ZtVlsI603k8-N1W1oOGWElmap-tGTq-i6QPa32BfZ_T-xH1ItzRxjItCm5FWET0PE51sEzAL1_oYE9uWwfeuI_MwPLFpA [following]
--2020-04-24 13:13:21--  https://ucc140f390f5f3e1ba8efe93669d.dl.dropboxusercontent.com/zip_download_get/AZ2B77jCErqrL0IC_ZtVlsI603k8-N1W1oOGWElmap-tGTq-i6QPa32BfZ_T-xH1ItzRxjItCm5FWET0P

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk import word_tokenize


In [0]:
filter=pd.read_csv(r'/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0',sep=',', encoding='latin-1')#,names=['ID','review','target'])

In [30]:
filter

,PK,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0.0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0.0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0.0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0.0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0.0
...,...,...,...
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0.0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0.0
1306121,ffffed09fedb5088744a,Who wins in a battle between a Wolverine and a...,0.0
1306122,PKÐÞ¾±´>g´>gPK,NaN,NaN


In [31]:
filter.shape

(1306124, 3)

In [32]:
filter.isnull().sum()

PK            0
question_text    2
target           2
dtype: int64

In [33]:
for col in ['question_text','target']:
  print(col,',',filter[col].value_counts())

question_text , From where maharatna PSUs hiring management trainees?                                                                                               1
Why is Islam a violent religion? Why all terrorists are all from Islam only? Why isn't Islam peaceful like Buddhism, Hinduism, Jainism, Sikhism?    1
How do Jeff bezos make money with Amazon?                                                                                                           1
How do I install AUX to a Suzuki Grand Vitara from 2006? Also possible to add 1-2 USB chargers?                                                     1
How do I know about a medicine being alien to the medical terms?                                                                                    1
                                                                                                                                                   ..
Is it hard to get a good job in Tasmania, Australia if you are a foreigner?         

In [0]:
filter.drop([1306122,1306123],0,inplace=True)

In [35]:
filter.isnull().sum()

PK            0
question_text    0
target           0
dtype: int64

In [36]:
filter.shape

(1306122, 3)

In [0]:
filter.drop(['PK'],1,inplace=True)

In [38]:
filter.dtypes

question_text     object
target           float64
dtype: object

In [0]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
#from keras.layers.recurrent import LSTM,GRU
#from keras.layers.core import Dense, Activation, Dropout
from sklearn.metrics import roc_auc_score, accuracy_score
#from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model
from nltk import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [0]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2020-04-24 12:40:43--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2020-04-24 12:40:43--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2020-04-24 12:40:43--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [0]:
!unzip /content/glove.42B.300d.zip 

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [40]:
!rm /content/glove.42B.300d.zip

rm: cannot remove '/content/glove.42B.300d.zip': No such file or directory


In [0]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()

In [51]:
len(embeding_index['provide'])

300

In [53]:
embeding_index['provide']

array([ 2.1902e-01, -6.6693e-02,  1.0538e-01, -2.0213e-01,  3.4797e-01,
       -8.4570e-03, -4.0350e+00,  3.4057e-01,  1.0484e-01,  2.1855e-01,
       -4.0837e-01,  2.7482e-01,  1.1953e-01,  7.4335e-02, -1.7069e-01,
       -4.0008e-01, -7.9120e-03, -3.9702e-01,  5.1159e-02, -2.3763e-02,
       -4.6144e-01,  4.7578e-02, -1.8110e-01,  3.9715e-03,  1.5718e-01,
       -1.8605e-01,  3.4680e-01,  1.7597e-01, -5.9390e-02, -3.0119e-01,
       -2.8326e-01, -2.6415e-01, -1.9358e-02,  6.7555e-03,  3.3796e-01,
       -3.5508e-01,  1.0336e-02, -5.0465e-01,  1.3932e-01, -2.7452e-01,
       -1.6377e-01,  5.0877e-01, -1.4303e-01, -2.0842e-01,  2.0463e-01,
        7.3660e-02, -1.8813e-01,  2.8356e-01, -2.7095e-01, -2.5136e-01,
       -1.2715e-01,  2.1164e-01, -4.4701e-01,  9.5515e-02,  4.5539e-01,
       -6.0036e-02,  1.4488e-02, -2.9433e-01, -5.5977e-01,  2.8814e-01,
        9.5750e-03,  2.2739e-01, -8.2802e-02,  3.0510e-01,  3.8501e-02,
        3.2902e-01, -1.1190e-02, -1.2909e-01,  3.6758e-01, -2.29

In [52]:
len(embeding_index)

1917494

In [0]:
X=filter['question_text']
Y=filter['target']

In [57]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1044897,), (261225,), (1044897,), (261225,))

In [58]:
class_weight=class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
print(class_weight)

[0.53310854 8.05092229]


In [59]:
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

{0: 0.5331085383324966, 1: 8.050922287457816}

In [60]:
x_train

595611                      How can I become a BMW mechanic?
1288453         What was the worst movie ending you've seen?
732783     Is someone who drives a lot more or less likel...
1028402     Why a fear stay when people started judging you?
433092     Which all CAT-T therapies are targeted for CD1...
                                 ...                        
955791     If you are not accountable after death, you ca...
54065      What are some useful tips for someone who is s...
724215                 What ingredients are in Marsala wine?
419695           What are the weirdest photos on your phone?
976039               Why isnât Hawkeye deaf in the movies?
Name: question_text, Length: 1044897, dtype: object

In [0]:
from nltk import word_tokenize

In [67]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [69]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

412


In [70]:
np.percentile(sent_lens,99)

44.0

In [0]:
max_len=44

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [79]:
tk.word_index

{'the': 1,
 'what': 2,
 'is': 3,
 'a': 4,
 'to': 5,
 'in': 6,
 'of': 7,
 'i': 8,
 'how': 9,
 'and': 10,
 'do': 11,
 'are': 12,
 'for': 13,
 'you': 14,
 'can': 15,
 'why': 16,
 'it': 17,
 'my': 18,
 'that': 19,
 'if': 20,
 'with': 21,
 'on': 22,
 'or': 23,
 'have': 24,
 'be': 25,
 'does': 26,
 'from': 27,
 'your': 28,
 'an': 29,
 'which': 30,
 'should': 31,
 'when': 32,
 'get': 33,
 'best': 34,
 'would': 35,
 'as': 36,
 'people': 37,
 'some': 38,
 'there': 39,
 'who': 40,
 'will': 41,
 'like': 42,
 'at': 43,
 'not': 44,
 'about': 45,
 'they': 46,
 'by': 47,
 'did': 48,
 'was': 49,
 'any': 50,
 'we': 51,
 'so': 52,
 'good': 53,
 'me': 54,
 'their': 55,
 'one': 56,
 'india': 57,
 'has': 58,
 'after': 59,
 'most': 60,
 'where': 61,
 'make': 62,
 'this': 63,
 'but': 64,
 'more': 65,
 'all': 66,
 'think': 67,
 'many': 68,
 'between': 69,
 'time': 70,
 'than': 71,
 'much': 72,
 'other': 73,
 'life': 74,
 'someone': 75,
 'use': 76,
 'he': 77,
 'way': 78,
 'out': 79,
 'am': 80,
 'know': 81,
 'u

In [80]:
vocab_size

195861

In [76]:
seq_train_matrix.shape

(1044897, 44)

In [75]:
seq_train

[[9, 15, 8, 93, 4, 5739, 10139],
 [2, 49, 1, 612, 296, 2780, 706, 561],
 [3, 75, 40, 4986, 4, 421, 65, 23, 309, 737, 5, 33, 6, 4, 2914],
 [16, 4, 903, 463, 32, 37, 668, 8578, 14],
 [30, 66, 567, 1112, 16007, 12, 5920, 13, 88029, 14181],
 [16, 11, 51, 23262, 697],
 [2, 3, 218, 3069, 10, 267, 562, 6, 227],
 [5399,
  9,
  72,
  581,
  41,
  25,
  780,
  47,
  4,
  98,
  2671,
  6110,
  10566,
  3692,
  511,
  8710],
 [16, 12, 38, 1410, 690, 21, 1, 14182, 7, 259, 32, 17, 1049, 12585, 13, 87],
 [15,
  51,
  25,
  2544,
  2652,
  20,
  51,
  95,
  76,
  4,
  4483,
  23,
  7403,
  23,
  3,
  39,
  50,
  73,
  996,
  197,
  30,
  51,
  15,
  25,
  16008,
  147,
  20,
  51,
  76,
  4,
  4483,
  23,
  7403],
 [2, 97, 25, 255, 47, 1166, 88030, 5, 5605, 55, 504, 218],
 [2, 12, 1, 60, 469, 817, 8352, 1087, 549, 623, 4610, 896],
 [9, 11, 8, 33, 4, 320, 70, 110, 6, 593, 1136, 21, 806],
 [9, 35, 14, 869, 1, 114, 7, 170, 132, 977, 8905, 20, 4282, 49, 2188, 20189],
 [15, 8, 76, 4, 696, 2568, 6517, 21, 6

In [77]:
x_train.shape

(1044897,)

In [88]:
len(seq_train),len(seq_test)

(1044897, 261225)

In [89]:
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 44), (261225, 44))

In [0]:
embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

In [91]:
 len(embedding_matrix)

195862

In [92]:
embedding_matrix.shape

(195862, 300)

In [93]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.20838   , -0.14932001, -0.017528  , ..., -0.54066002,
         0.21199   , -0.0094357 ],
       [ 0.13877   , -0.0021201 , -0.091039  , ...,  0.024614  ,
         0.46514001,  0.11363   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.020497  ,  0.053071  ,  0.52139997, ..., -0.45282   ,
        -0.33327001, -0.28336999],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [0]:
inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [96]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 44)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 44, 300)           58758600  
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               3289

In [0]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)

In [0]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [106]:
model.fit(seq_train_matrix,y_train,
              epochs=10,
              class_weight={0: 0.5331085383324966, 1: 8.050922287457816},
              batch_size=10000,
              validation_data=(seq_test_matrix,y_test),
              callbacks = [earlystop, checkpoint])

Train on 1044897 samples, validate on 261225 samples
Epoch 1/10
1044897/1044897 [==============================] - 102s 98us/step - loss: 0.1905 - accuracy: 0.9069 - val_loss: 0.1959 - val_accuracy: 0.9152

Epoch 00001: val_loss improved from 0.20928 to 0.19595, saving model to /content/weights-01-0.1959.h5
Epoch 2/10
1044897/1044897 [==============================] - 102s 98us/step - loss: 0.1805 - accuracy: 0.9093 - val_loss: 0.2152 - val_accuracy: 0.9076

Epoch 00002: val_loss did not improve from 0.19595
Epoch 3/10
1044897/1044897 [==============================] - 102s 98us/step - loss: 0.1710 - accuracy: 0.9132 - val_loss: 0.2280 - val_accuracy: 0.9015

Epoch 00003: val_loss did not improve from 0.19595
Epoch 4/10
1044897/1044897 [==============================] - 102s 98us/step - loss: 0.1604 - accuracy: 0.9164 - val_loss: 0.1931 - val_accuracy: 0.9143

Epoch 00004: val_loss improved from 0.19595 to 0.19310, saving model to /content/weights-04-0.1931.h5
Epoch 5/10
1044897/104489

In [0]:
best_model = load_model('/content/weights-01-0.1959.h5')

In [108]:
best_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 44)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 44, 300)           58758600  
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               3289

In [109]:
p = best_model.predict(seq_test_matrix)
roc_auc_score(y_test,p)

0.9639427638764951